In [1]:
!pip install gdown
!gdown --id 1aUTHbMopudfj_ATmmp_DNjr2BxX9M1c8
!gdown --id 1X5Ek5mQYjNkw8k87gt_VCFDhPCSwXbmF
!gdown --id 1q_T2QobAT3EGdiDIuZ35qIGsDY3ynzLg

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1aUTHbMopudfj_ATmmp_DNjr2BxX9M1c8
From (redirected): https://drive.google.com/uc?id=1aUTHbMopudfj_ATmmp_DNjr2BxX9M1c8&confirm=t&uuid=74e45c99-c0e9-4790-9396-b7a03c8f10b7
To: /kaggle/working/3D-IR-MAH-CT.zip
100%|███████████████████████████████████████| 2.73G/2.73G [00:21<00:00, 127MB/s]
/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1X5Ek5mQYjNkw8k87gt_VCFDhPCSwXbmF
From (redirected): https://drive.google.com/uc?id=1X5Ek5mQYjNkw8k87gt_VCFDhPCSwXbmF&confirm=t&uuid=e50c7d88-7875-4ab8-82

In [ ]:
!unzip '/kaggle/working/NL3DTIF.zip' -d '/kaggle/working/'
!unzip '/kaggle/working/3D-IR-MAH-CT.zip' -d '/kaggle/working/'

In [3]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

from scipy import ndimage

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
os.remove('/kaggle/working/3D-IR-MAH-CT.zip')
os.remove('/kaggle/working/NL3DTIF.zip')

In [5]:
def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 128
    desired_width = 128
    desired_height = 128
    # Get current depth
    current_depth = img.shape[0]
    current_width = img.shape[1]
    current_height = img.shape[2]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Resize across z-axis
    img = ndimage.zoom(img, (depth_factor, width_factor, height_factor, 1))
    return img






def plot_slices(num_rows, num_cols, data):
    # create figure
    fig = plt.figure(figsize=(10, 7))
    
    n = len(data)
  
    for i in range(num_rows * num_cols):
        if i < n:
            # Adds a subplot at the 1st position
            fig.add_subplot(num_rows, num_cols, i+1)

            # showing image
            plt.imshow(data[i])
            plt.axis('off')
            
            
      
    
    
def prepare_3D_samples(main_dir, inp_lst):
    
    final_lst = []
    for i in tqdm(inp_lst):
        tmp_dir = os.path.join(main_dir,i)
        tmp_lst = os.listdir(tmp_dir)
        tmp_lst.sort()
        
        tmp_3d = []
        for j in tmp_lst:
            tmp_3d.append(plt.imread(os.path.join(main_dir,i,j)))
        
        
        final_lst.append(resize_volume(np.array(tmp_3d)))
    
    return np.array(final_lst)




def rotate(volume):
    """Rotate the volume by a few degrees"""

    def scipy_rotate(volume):
        # define some rotation angles
        angles = [-20, -10, -5, 0, 5, 10, 20]
        # pick angles at random
        angle = random.choice(angles)
        # rotate volume
        volume = ndimage.rotate(volume, angle, reshape=False)
        return volume
    
    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32)
    return augmented_volume


In [6]:
cov_main_dir = '/kaggle/working/covid_3D'
norm_main_dir = '/kaggle/working/NL3DTIF'

cov_lst = os.listdir(cov_main_dir)
cov_lst.sort()
random.Random(1).shuffle(cov_lst)

norm_lst = os.listdir(norm_main_dir)
norm_lst.sort()
random.Random(1).shuffle(norm_lst)

print(len(cov_lst))
print(cov_lst[:5])
print(len(norm_lst))
print(norm_lst[:5])
print('_________')

covid_volume = prepare_3D_samples(cov_main_dir, cov_lst)
normal_volume = prepare_3D_samples(norm_main_dir, norm_lst)
print('_________')

print('covid_volume:', np.shape(covid_volume))
print('normal_volume:', np.shape(normal_volume))

133
['77', '84', '65', '61', '97']
76
['020', '067', '003', '057', '012']
_________


100%|██████████| 76/76 [37:25<00:00, 29.54s/it]


_________
covid_volume: (133, 128, 128, 128, 3)
normal_volume: (76, 128, 128, 128, 3)


In [7]:
normal_labels = np.array([1 for _ in range(len(normal_volume))])
covid_labels = np.array([0 for _ in range(len(covid_volume))])

In [8]:
x_train = np.concatenate((covid_volume[:107], normal_volume[:61]), axis=0)
y_train = np.concatenate((covid_labels[:107], normal_labels[:61]), axis=0)

x_valid = np.concatenate((covid_volume[107:120], normal_volume[61:68]), axis=0)
y_valid = np.concatenate((covid_labels[107:120], normal_labels[61:68]), axis=0)

x_test = np.concatenate((covid_volume[120:], normal_volume[68:]), axis=0)
y_test = np.concatenate((covid_labels[120:], normal_labels[68:]), axis=0)

In [9]:
print('x_train:', np.shape(x_train))
print('y_train:', np.shape(y_train))
print('x_valid:', np.shape(x_valid))
print('y_valid:', np.shape(y_valid))
print('x_test:', np.shape(x_test))
print('y_test:', np.shape(y_test))

x_train: (168, 128, 128, 128, 3)
y_train: (168,)
x_valid: (20, 128, 128, 128, 3)
y_valid: (20,)
x_test: (21, 128, 128, 128, 3)
y_test: (21,)


In [10]:
def train_preprocessing(volume, label):
    # rescaling
    volume = tf.cast(volume, tf.float32) / 255.0
    # Rotate volume
    volume = rotate(volume)
    return volume, label


def validation_preprocessing(volume, label):
    # rescaling
    volume = tf.cast(volume, tf.float32) / 255.0
    return volume, label

In [11]:
# Define data loaders.
train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
validation_loader = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
test_loader = tf.data.Dataset.from_tensor_slices((x_test, y_test))

batch_size = 2

# Augment the on the fly during training.
train_dataset = (
    train_loader.shuffle(len(x_train))
    .map(train_preprocessing)
    .batch(batch_size)
    .prefetch(1)
)

# Only rescale.
validation_dataset = (
    validation_loader.shuffle(len(x_valid))
    .map(validation_preprocessing)
    .batch(batch_size)
    .prefetch(1)
)

# Only rescale.
test_dataset = (
    test_loader.shuffle(len(x_test))
    .map(validation_preprocessing)
    .batch(batch_size)
    .prefetch(1)
)

# ablation

In [13]:
def maseres():
    model = tf.keras.models.load_model('/kaggle/working/MASERes_3.h5')
    return model

def ablation(inp_model, inp_layer, inp_filters):
    # inp_model: the model on which ablation is applied
    # inp_layer: an integer that shows the layer's order (n: n'th layer)
    # inp_filter: a list of integers that shows the filter's
    #                     order in the layer([a,b,c]: a'th, b'th, and c'th filters)
    if len(inp_model.layers[inp_layer].get_weights())  == 2:
        w = inp_model.layers[inp_layer].get_weights()[0]
        b = inp_model.layers[inp_layer].get_weights()[1]
        for i in inp_filters:
            w[..., i] = np.zeros((np.shape(w[... ,i])))
            b[i] = 0
        inp_model.layers[inp_layer].set_weights([w,b])
      
    
    elif len(inp_model.layers[inp_layer].get_weights()) == 1:
        w = inp_model.layers[inp_layer].get_weights()[0]
        for i in inp_filters:
                w[..., i] = np.zeros((np.shape(w[... ,i])))
        inp_model.layers[inp_layer].set_weights([w])
    
    return inp_model


def ablation_attention(inp_model, inp_layer, inp_neurons):
    
    w = inp_model.layers[inp_layer].get_weights()[0]
    b = inp_model.layers[inp_layer].get_weights()[1]

    for i in inp_neurons:
        w[i] = np.zeros((np.shape(w[i])))
    inp_model.layers[inp_layer].set_weights([w, b])
    
    return inp_model


def results(inp_model, inp_data):
    inp_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6) , 
                loss='binary_crossentropy', metrics = ['acc',
                                                      tf.keras.metrics.AUC(),
                                                      tf.keras.metrics.Precision(),
                                                      tf.keras.metrics.Recall(),
                                                      tf.keras.metrics.TruePositives(),
                                                      tf.keras.metrics.TrueNegatives(),
                                                      tf.keras.metrics.FalsePositives(),
                                                      tf.keras.metrics.FalseNegatives()])
    
    a = inp_model.evaluate(inp_data)

    conf_mx = [[a[6], a[7]],[a[8], a[5]]]
    mx = np.array(conf_mx)
    fscore = 2*a[3]*a[4]/(a[3]+a[4])
    spc = mx[0, 0] * 1.0 / (mx[0, 0] + mx[0, 1])
    sen = mx[1,1] * 1.0 / (mx[1,1] + mx[1,0])

    print('_________')
    print('accuracy:',np.round(a[1]*100,2),'%')
    print('precision:',np.round(a[3]*100,2),'%')
    print('recall:',np.round(a[4]*100,2),'%')
    print('Sensitivity:',np.round(sen*100,2),'%')
    print('Specificity:',np.round(spc*100,2),'%')
    print('f1-score:',np.round(fscore*100,2),'%')
    print('AUC:',np.round(a[2]*100,2),'%')

In [14]:
# main model's layers

main_model = maseres()
for i, j in enumerate(main_model.layers):
    print('idx', i, ':', j)

idx 0 : <keras.engine.input_layer.InputLayer object at 0x7e6096bb3970>
idx 1 : <keras.layers.convolutional.conv3d.Conv3D object at 0x7e6096bb22f0>
idx 2 : <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7e6096bb3460>
idx 3 : <keras.layers.core.tf_op_layer.TFOpLambda object at 0x7e6096bb3070>
idx 4 : <keras.layers.convolutional.conv3d.Conv3D object at 0x7e6096b99d50>
idx 5 : <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7e6096b99ae0>
idx 6 : <keras.layers.core.tf_op_layer.TFOpLambda object at 0x7e6096b99510>
idx 7 : <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7e6096b99000>
idx 8 : <keras.layers.core.tf_op_layer.TFOpLambda object at 0x7e6096b98d30>
idx 9 : <keras.layers.convolutional.conv3d.Conv3D object at 0x7e6096b98880>
idx 10 : <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7e6096b983d0>
idx 11 : <keras.layers.core.tf_op_layer.TFOpLambda object at 0x7e609

In [16]:
# Damaged Model I

main_model = maseres()

# index of filters to ablate from layer idx1 (selecting 4 out of 16)
tmp_lst1 = list(range(16))
random.Random(1).shuffle(tmp_lst1)
idx1_ablated_filters = tmp_lst1[:4]

# index of filters to ablate from layer idx4 (selecting 4 out of 16)
tmp_lst2 = list(range(16))
random.Random(2).shuffle(tmp_lst2)
idx2_ablated_filters = tmp_lst2[:4]

# ablation in layer idx1
tmp_dmg_i_model = ablation(main_model, inp_layer = 1, inp_filters = idx1_ablated_filters)
damaged_model_i = ablation(tmp_dmg_i_model, inp_layer = 4,
                           inp_filters = idx2_ablated_filters)

results(damaged_model_i, test_dataset)

11/11 [==============================] - 14s 446ms/step - loss: 0.8562 - acc: 0.4762 - auc: 0.9904 - precision: 0.4211 - recall: 1.0000 - true_positives: 8.0000 - true_negatives: 2.0000 - false_positives: 11.0000 - false_negatives: 0.0000e+00
_________
accuracy: 47.62 %
precision: 42.11 %
recall: 100.0 %
Sensitivity: 100.0 %
Specificity: 15.38 %
f1-score: 59.26 %
AUC: 99.04 %


In [17]:
# Damaged Model II

main_model = maseres()

layers_lst = [9,12,14,15,17,21,24,26,27,29,33,36,38,39,41]
idx = dict()
for i, lyr in enumerate(layers_lst):
    n_filters = np.shape(main_model.layers[lyr].get_weights()[0])
    idx[f'{lyr}'] = list(range(n_filters[-1]))
    random.Random(i+1).shuffle(idx[f'{lyr}'])
    idx[f'{lyr}'] = idx[f'{lyr}'][:int(0.25*n_filters[-1])]
    
for k,v in idx.items():    
    damaged_model_ii = ablation(main_model, inp_layer = int(k), inp_filters = v)
    
results(damaged_model_ii, test_dataset)

11/11 [==============================] - 5s 330ms/step - loss: 0.3380 - acc: 0.8571 - auc_1: 0.9712 - precision_1: 1.0000 - recall_1: 0.6250 - true_positives_1: 5.0000 - true_negatives_1: 13.0000 - false_positives_1: 0.0000e+00 - false_negatives_1: 3.0000
_________
accuracy: 85.71 %
precision: 100.0 %
recall: 62.5 %
Sensitivity: 62.5 %
Specificity: 100.0 %
f1-score: 76.92 %
AUC: 97.12 %


In [21]:
# Damaged Model III

main_model = maseres()

layers_lst = [44,45,46,47,48,49,54,56,58]
idx = dict()
for i, lyr in enumerate(layers_lst):
    n_neuron = np.shape(main_model.layers[lyr].get_weights()[0])
    idx[f'{lyr}'] = list(range(n_neuron[0]))
    random.Random(i+1).shuffle(idx[f'{lyr}'])
    idx[f'{lyr}'] = idx[f'{lyr}'][:int(0.25*n_neuron[0])]
    
for k,v in idx.items():    
    damaged_model_iii = ablation_attention(main_model, inp_layer = int(k), inp_neurons = v)
    
damaged_model_iii.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6) , 
                loss='binary_crossentropy', metrics = ['acc',
                                                      tf.keras.metrics.AUC(),
                                                      tf.keras.metrics.Precision(),
                                                      tf.keras.metrics.Recall(),
                                                      tf.keras.metrics.TruePositives(),
                                                      tf.keras.metrics.TrueNegatives(),
                                                      tf.keras.metrics.FalsePositives(),
                                                      tf.keras.metrics.FalseNegatives()])

damaged_model_iii.evaluate(test_dataset)

11/11 [==============================] - 5s 328ms/step - loss: 1.9398 - acc: 0.6190 - auc_5: 0.7500 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00 - true_positives_5: 0.0000e+00 - true_negatives_5: 13.0000 - false_positives_5: 0.0000e+00 - false_negatives_5: 8.0000


[1.9398425817489624, 0.6190476417541504, 0.75, 0.0, 0.0, 0.0, 13.0, 0.0, 8.0]

In [19]:
# Damaged Model IV

main_model = maseres()

layers_lst = [64]
idx = dict()
for i, lyr in enumerate(layers_lst):
    n_filters = np.shape(main_model.layers[lyr].get_weights()[0])
    idx[f'{lyr}'] = list(range(n_filters[-1]))
    random.Random(i+1).shuffle(idx[f'{lyr}'])
    idx[f'{lyr}'] = idx[f'{lyr}'][:int(0.25*n_filters[-1])]
    
for k,v in idx.items():    
    damaged_model_iv = ablation(main_model, inp_layer = int(k), inp_filters = v)
    
results(damaged_model_iv, test_dataset)

11/11 [==============================] - 5s 329ms/step - loss: 0.1465 - acc: 0.9524 - auc_3: 1.0000 - precision_3: 1.0000 - recall_3: 0.8750 - true_positives_3: 7.0000 - true_negatives_3: 13.0000 - false_positives_3: 0.0000e+00 - false_negatives_3: 1.0000
_________
accuracy: 95.24 %
precision: 100.0 %
recall: 87.5 %
Sensitivity: 87.5 %
Specificity: 100.0 %
f1-score: 93.33 %
AUC: 100.0 %


In [20]:
# Damaged Model V

main_model = maseres()

layers_lst = [65]
idx = dict()
for i, lyr in enumerate(layers_lst):
    n_filters = np.shape(main_model.layers[lyr].get_weights()[0])
    idx[f'{lyr}'] = list(range(n_filters[-1]))
    random.Random(i+1).shuffle(idx[f'{lyr}'])
    idx[f'{lyr}'] = idx[f'{lyr}'][:int(0.25*n_filters[-1])]
    
for k,v in idx.items():    
    damaged_model_v = ablation(main_model, inp_layer = int(k), inp_filters = v)
    
results(damaged_model_v, test_dataset)

11/11 [==============================] - 5s 329ms/step - loss: 0.0807 - acc: 1.0000 - auc_4: 1.0000 - precision_4: 1.0000 - recall_4: 1.0000 - true_positives_4: 8.0000 - true_negatives_4: 13.0000 - false_positives_4: 0.0000e+00 - false_negatives_4: 0.0000e+00
_________
accuracy: 100.0 %
precision: 100.0 %
recall: 100.0 %
Sensitivity: 100.0 %
Specificity: 100.0 %
f1-score: 100.0 %
AUC: 100.0 %
